In [2]:
#Importing the neccessary modules
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
#the training and testing datasets in csv format
train_data = "fashion-mnist_train.csv"
test_data = "fashion-mnist_test.csv"

In [6]:
#reading the train_data and test_data usign the pandas libary
train = pd.read_csv(train_data)
test = pd.read_csv(test_data)

In [7]:
print("Test\n\n", train.head() , "\n\n")
print("Train\n\n" , test.head() , "\n\n")

Test

    label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0        30        43         0   
3       0  ...         3         0         0         0         0         1   
4       0  ...         0         0         0         0         0         0   

   pixel781  pixel782  pixel783  pixel784  
0         0         0  

In [8]:
#function to get the features and labels from the train and test data
def get_features_labels(data):
    features = data.values[:, 1:]/255
    labels = data['label'].values
    return features, labels

In [90]:
#getting the features and labels 
train_features, train_labels = get_features_labels(train)
test_features, test_labels = get_features_labels(test)

#printing their shapes
print(train_features.shape)
print(test_features.shape)


(60000, 784)
(10000, 784)


<h4>Building the keras model</h4>

In [11]:
# Building the keras model with the relu and softmax activation fuctions
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

#Compiling the model and printing the summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 30)                23550     
_________________________________________________________________
dense_4 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
Total params: 24,380
Trainable params: 24,380
Non-trainable params: 0
_________________________________________________________________


In [12]:
#encoding the labels becuase i am using categorical_crossentropy in compiling the model
train_labels_1hot = tf.keras.utils.to_categorical(train_labels)
test_labels_1hot = tf.keras.utils.to_categorical(test_labels)

model.fit(train_features, train_labels_1hot, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 15s 247us/sample - loss: 0.6508 - acc: 0.7759
Epoch 2/5
60000/60000 [==============================] - 29s 476us/sample - loss: 0.4572 - acc: 0.8397
Epoch 3/5
60000/60000 [==============================] - 13s 214us/sample - loss: 0.4152 - acc: 0.8545
Epoch 4/5
60000/60000 [==============================] - 12s 199us/sample - loss: 0.3902 - acc: 0.8641
Epoch 5/5
60000/60000 [==============================] - 13s 223us/sample - loss: 0.3738 - acc: 0.8676 - loss: 0.3743 - acc: 0.86 - ETA: 2s


In [13]:
#evaluate the model and print its loss and accuracy
test_loss, test_accuracy = model.evaluate(test_features, test_labels_1hot)

print("Test Loss = ", test_loss)
print("Test accuracy = ", test_accuracy)

10000/10000 [==============================] - 2s 160us/sample - loss: 0.3869 - acc: 0.8600
Test Loss =  0.38687515287399293
Test accuracy =  0.86


In [14]:
#Then i save the model trained with keras
model.save('keras_model.h5')

<h4>Converting the model to tf.estimator model</h4>

In [92]:
#converting the keras model to estimator model
tf_classifier_model = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Javis\\AppData\\Local\\Temp\\tmpvterx6ar', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000EB63AB1688>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


<h4>Training the model with tf.estimator</h4>

In [93]:
input_name = model.input_names[0]

#preparing input for training the model with tf.estimator
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={input_name: train_features}, y=train_labels_1hot, batch_size=128, 
                                                    num_epochs=5, shuffle=True)
#getting input names
feature_columns = [tf.feature_column.numeric_column(input_name, shape=784)]

In [94]:
classifier = tf_classifier_model

In [95]:
#training the model
classifier.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='C:\\Users\\Javis\\AppData\\Local\\Temp\\tmpvterx6ar\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('C:\\Users\\Javis\\AppData\\Local\\Temp\\tmpvterx6ar\\keras\\keras_model.ckpt',)
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 6 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Javis\AppData\Local\Temp\tmpvterx6ar\model.ckpt.
INFO:tensorflow:loss = 0.38186187, step = 1
INFO:tensorflow:global_step/sec: 45.9532
INFO:tensorflow:loss = 0.43143943, step = 101 (2.181 sec)
INFO:tensorflow:global_step/s

In [96]:
#Evaluating the model

evaluate_input = tf.estimator.inputs.numpy_input_fn(x={input_name: test_features}, y=test_labels_1hot, num_epochs=1, 
                                                    shuffle=False)

classifier.evaluate(input_fn=evaluate_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-07T03:48:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Javis\AppData\Local\Temp\tmpvterx6ar\model.ckpt-2344
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-07-03:48:02
INFO:tensorflow:Saving dict for global step 2344: acc = 0.8713, global_step = 2344, loss = 0.36308756
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2344: C:\Users\Javis\AppData\Local\Temp\tmpvterx6ar\model.ckpt-2344


{'acc': 0.8713, 'loss': 0.36308756, 'global_step': 2344}

In [98]:
#save the model trianed with the estimator
feature_spec = {
    input_name: tf.FixedLenFeature(shape=[784], dtype=np.float32)
}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel(export_dir_base="", 
                            serving_input_receiver_fn=serving_fn)
export_dir = export_dir.decode("utf8")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\Javis\AppData\Local\Temp\tmpvterx6ar\model.ckpt-2344
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: temp-b'1575688012'\saved_model.pb


<p>From my observation, I saw that the tf.keras model trained faster than that of the tf.estimator model but both gave similar accuracy ouput that is 86% and 87% respectively. For me, a tf,keras model is mor easy to build than a tf.estimator model. <br />
    The two API's are similar in some ways like the both have the evaluate() function but the also differ in some ways such as the Sequential() class in tf.keras that is not present in tf.estimator also the train() function in tf.estimator is not found in tf.keras</p>